# Importing Necessary Libraries

In [173]:
import pandas as pd
import  re
import numpy as np

In [174]:
# Reading CSV file
df = pd.read_csv("./Job_Dataset.csv")

In [175]:
df.head(2)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
0,United States of America,US,NaN,No,jobs.monster.com,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,Full Time Employee,"Madison, WI 53702",NaN,http://jobview.monster.com/it-support-technici...,NaN,IT/Software Development,11d599f229a80023d2f40e7c52cd941e
1,United States of America,US,NaN,No,jobs.monster.com,The Wisconsin State Journal is seeking a flexi...,Business Reporter/Editor Job in Madison,Full Time,"Madison, WI 53708",Printing and Publishing,http://jobview.monster.com/business-reporter-e...,NaN,NaN,e4cbb126dabf22159aff90223243ff2a


#### [1] Giving Appropiate DataType To Date_added

In [176]:
df.date_added.dtypes

dtype('O')

In [177]:
df['date_added'] = pd.to_datetime(df['date_added'])

In [178]:
df.date_added.unique()
# note : Time values will be removed in following steps in date_added cleaning 

array([                          'NaT', '2016-05-10T00:00:00.000000000',
       '2016-05-13T00:00:00.000000000', '2016-05-09T00:00:00.000000000',
       '2016-06-10T00:00:00.000000000', '2017-01-02T00:00:00.000000000',
       '2016-06-17T00:00:00.000000000', '2016-06-24T00:00:00.000000000',
       '2016-06-23T00:00:00.000000000', '2016-12-20T00:00:00.000000000',
       '2016-12-12T00:00:00.000000000', '2016-07-15T00:00:00.000000000',
       '2016-07-08T00:00:00.000000000', '2016-10-31T00:00:00.000000000',
       '2016-10-28T00:00:00.000000000', '2016-04-28T00:00:00.000000000',
       '2016-04-27T00:00:00.000000000', '2016-04-29T00:00:00.000000000',
       '2016-09-12T00:00:00.000000000', '2017-01-16T00:00:00.000000000',
       '2016-09-22T00:00:00.000000000', '2016-03-25T00:00:00.000000000',
       '2016-06-20T00:00:00.000000000', '2016-11-17T00:00:00.000000000',
       '2016-07-11T00:00:00.000000000', '2016-12-07T00:00:00.000000000',
       '2016-12-14T00:00:00.000000000', '2016-12-29

# Cleaning job_title Column

#### Some rows of column Job_title have the title after substring "Job Application for" while others have title value before 'job'substring

## [1] Extracting String after "Job Application for" 

In [179]:
# Fetching Rows That Contains Substring "Job Application for"
df[df['job_title'].str.contains("Job Application for")].head(2)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
2,United States of America,US,NaT,No,jobs.monster.com,Report this job About the Job DePuy Synthes Co...,Johnson & Johnson Family of Companies Job Appl...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,http://jobview.monster.com/senior-training-lea...,NaN,NaN,839106b353877fa3d896ffb9c1fe01c0
6,United States of America,US,NaT,No,jobs.monster.com,Report this job About the Job Based in San Fra...,CyberCoders Job Application for Principal QA E...,"Full Time, Employee",Contact name Tony Zerio,Computer Software,http://jobview.monster.com/principal-qa-engine...,NaN,NaN,455802d725fde67293970ab3953b1d39


In [180]:
# Spliting job_title into two columns across Substring "Job Application for"
df[['job_title','job_title_2']] = df['job_title'].str.split("Job Application for",expand = True)

In [181]:
df[['job_title','job_title_2']].head(3)

,job_title,job_title_2
0,IT Support Technician Job in Madison,None
1,Business Reporter/Editor Job in Madison,None
2,Johnson & Johnson Family of Companies,Senior Training Leader | Monster.com var MONS...


In [182]:
# add title from job_title_2 to job_title
# wherever job_title_2 is not null copy (mask) that non null value into job_title column
df['job_title'] = df['job_title'].mask(df['job_title_2'].notnull(),df['job_title_2'],axis = 0)

In [183]:
df[['job_title','job_title_2']].head(5)

,job_title,job_title_2
0,IT Support Technician Job in Madison,None
1,Business Reporter/Editor Job in Madison,None
2,Senior Training Leader | Monster.com var MONS...,Senior Training Leader | Monster.com var MONS...
3,Engineer - Quality Job in Dixon,None
4,Shift Supervisor - Part-Time Job in Camphill,None


In [184]:
df.drop("job_title_2",axis=1,inplace = True)

## [2] Removing unwanted text like ' | Monster.com var MONS..' from such titles

In [185]:
df['job_title'].str.split("|",expand = True).head(10)

,0,1,2,3
0,IT Support Technician Job in Madison,None,None,None
1,Business Reporter/Editor Job in Madison,None,None,None
2,Senior Training Leader,"Monster.com var MONS_LOG_VARS = {""JobID"":",None,None
3,Engineer - Quality Job in Dixon,None,None,None
4,Shift Supervisor - Part-Time Job in Camphill,None,None,None
5,Construction PM - Charlottesville Job in Charl...,None,None,None
6,Principal QA Engineer - Java,None,None,None
7,Mailroom Clerk Job in Austin,None,None,None
8,Housekeeper Job in Austin,None,None,None
9,Video Data Management /Transportation Technici...,None,None,None


In [186]:
df[['job_title','job_dummy']] = df['job_title'].str.split("|",n=1,expand = True)

In [187]:
df[['job_title','job_dummy']].head(10)

,job_title,job_dummy
0,IT Support Technician Job in Madison,None
1,Business Reporter/Editor Job in Madison,None
2,Senior Training Leader,"Monster.com var MONS_LOG_VARS = {""JobID"":"
3,Engineer - Quality Job in Dixon,None
4,Shift Supervisor - Part-Time Job in Camphill,None
5,Construction PM - Charlottesville Job in Charl...,None
6,Principal QA Engineer - Java,None
7,Mailroom Clerk Job in Austin,None
8,Housekeeper Job in Austin,None
9,Video Data Management /Transportation Technici...,None


In [188]:
df.drop("job_dummy",axis=1,inplace = True)

## [3]  Everything After  Substring 'Job' In JobTitle Is Already In  Location Column 
So, removing everyting after substring 'Job'

In [189]:
df[['job_title','location']].head(2)

,job_title,location
0,IT Support Technician Job in Madison,"Madison, WI 53702"
1,Business Reporter/Editor Job in Madison,"Madison, WI 53708"


In [190]:
df.job_title.str.split('Job',n=1,expand=True).head(2)

,0,1
0,IT Support Technician,in Madison
1,Business Reporter/Editor,in Madison


In [191]:
df[['job_title','location_dummy']] = df.job_title.str.split('Job',n=1,expand=True) 

i had to create location_dummy(which i will drop later) as str.spit resulted to 2 columns itself

In [192]:
df.drop("location_dummy",axis=1,inplace = True)

In [193]:
df.head(4)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
0,United States of America,US,NaT,No,jobs.monster.com,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician,Full Time Employee,"Madison, WI 53702",NaN,http://jobview.monster.com/it-support-technici...,NaN,IT/Software Development,11d599f229a80023d2f40e7c52cd941e
1,United States of America,US,NaT,No,jobs.monster.com,The Wisconsin State Journal is seeking a flexi...,Business Reporter/Editor,Full Time,"Madison, WI 53708",Printing and Publishing,http://jobview.monster.com/business-reporter-e...,NaN,NaN,e4cbb126dabf22159aff90223243ff2a
2,United States of America,US,NaT,No,jobs.monster.com,Report this job About the Job DePuy Synthes Co...,Senior Training Leader,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,http://jobview.monster.com/senior-training-lea...,NaN,NaN,839106b353877fa3d896ffb9c1fe01c0
3,United States of America,US,NaT,No,jobs.monster.com,Why Join Altec? If you’re considering a career...,Engineer - Quality,Full Time,"Dixon, CA",Altec Industries,http://jobview.monster.com/engineer-quality-jo...,NaN,Experienced (Non-Manager),58435fcab804439efdcaa7ecca0fd783


## [4] Part-Time, Full-Time Info In Job_Title Is Already Available In Job_Type
So, Removing Info From Job_Title Which Is Already In Job_Type

In [194]:
df[df['job_title'].str.contains("Part-Time")].head(2)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
4,United States of America,US,NaT,No,jobs.monster.com,Position ID# 76162 # Positions 1 State CT C...,Shift Supervisor - Part-Time,Full Time Employee,"Camphill, PA",Retail,http://jobview.monster.com/shift-supervisor-pa...,NaN,Project/Program Management,64d0272dc8496abfd9523a8df63c184c
102,United States of America,US,NaT,No,jobs.monster.com,The Customer Service Representative (CSR) is r...,Customer Service Representative (Part-Time) / ...,Part Time Employee,"Lansing, MI 48911","Travel, Transportation and Tourism",http://jobview.monster.com/Customer-Service-Re...,NaN,Customer Support/Client Care,ed3a0136b8c7b78144c18124d9d3263f


In [195]:
df[df['job_title'].str.contains("Full-Time")].head(2)


,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
271,United States of America,US,NaT,No,jobs.monster.com,Report this job About the Job Work Schedule: ...,Non-CDL Route Driver (Full-Time),"Full Time, Employee",Work Schedule: Daily route schedules dete...,Transport and Storage - Materials,http://jobview.monster.com/non-cdl-route-drive...,NaN,NaN,6c2454474099e8fee3768830c109668f
319,United States of America,US,NaT,No,jobs.monster.com,Dietary Aide Full-Time ( Job Number: 298216 ) ...,Dietary Aide Full-Time,"Full Time, Employee",Dietary Aide Full-Time ( Job Number: 298216 ) ...,Healthcare Services,http://jobview.monster.com/dietary-aide-full-t...,NaN,Food Services/Hospitality,91a189a04cbd193af2f136e9fc837fae


In [196]:
df[df['job_title'].str.contains("Part Time")].head(2)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
302,United States of America,US,NaT,No,jobs.monster.com,DescriptionSurprise. Delight. Engage. Amaze. A...,Macy's **Seasonal Holiday Retail Sales** Part ...,Full Time Employee,"Dallas, TX",NaN,http://jobview.monster.com/macy's-seasonal-hol...,NaN,Sales/Retail/Business Development,73b3e1df114ed0c299b598b2cc5300a8
346,United States of America,US,NaT,No,jobs.monster.com,Summary RETAIL RESET MERCHANDISER KROGER PART ...,Retail Reset Merchandiser Kroger Part Time,Part Time Employee,"Las Vegas, NV",Retail,http://jobview.monster.com/retail-reset-mercha...,NaN,Sales/Retail/Business Development,bb9b4a486a27e61549d6ed5717fd049f


In [197]:
df[df['job_title'].str.contains("Full Time")].head(2)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
54,United States of America,US,NaT,No,jobs.monster.com,Retail Customer Engagement Specialist- Entry L...,Retail Merchandise Apprenticeship- Full Time &...,Full Time Employee,"Houston, TX 77040",RetailAdvertising and PR ServicesBusiness Serv...,http://jobview.monster.com/Retail-Merchandise-...,NaN,Marketing/Product,4333b6abfff1445594a05884dc79950b
96,United States of America,US,NaT,No,jobs.monster.com,Financial Advisor Northwestern MutualOur finan...,Financial Advisor / Financial Sales Representa...,Full Time Employee,"Houston, TX",Banking,http://jobview.monster.com/Financial-Advisor-F...,NaN,Sales/Retail/Business Development,e52d5e30ecdf97dd64a44f45bfc2032b


#### This is will Replace above substrings with blanks ("")

In [198]:
df['job_title'] = df["job_title"].str.replace(r'((?i)full[ -]time|(?i)part[ -]time|(?i)per[ -]diem)','',regex=True)

In [199]:
df.job_title

0                                IT Support Technician 
1                             Business Reporter/Editor 
2                               Senior Training Leader 
3                                   Engineer - Quality 
4                                  Shift Supervisor -  
                              ...                      
21995            Assistant Vice President - Controller 
21996                                       Accountant 
21997                                 AEM/CQ developer 
21998    Electrician - Experienced Forging Electrician 
21999                           Contract Administrator 
Name: job_title, Length: 22000, dtype: object

#### Removing rows with () 

In [200]:
df[df['job_title'].str.contains('\(\)')].head(2)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
99,United States of America,US,NaT,No,jobs.monster.com,"MOUNTAIN, LTD. is currently seeking an OSP Fie...",Outside Plant Telecom Engineer (),Full Time,"Fort Collins, CO",NaN,http://jobview.monster.com/Outside-Plant-Telec...,NaN,NaN,fbabdfd6692540e30d7bccfe393adabe
102,United States of America,US,NaT,No,jobs.monster.com,The Customer Service Representative (CSR) is r...,Customer Service Representative () / CSR,Part Time Employee,"Lansing, MI 48911","Travel, Transportation and Tourism",http://jobview.monster.com/Customer-Service-Re...,NaN,Customer Support/Client Care,ed3a0136b8c7b78144c18124d9d3263f


In [201]:
df[df['job_title'].str.contains('\( . \)')].head(2)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
47,United States of America,US,NaT,No,jobs.monster.com,The Project Manager reports directly to the Co...,Project Manager ( Z ),Full Time Employee,"Houston, TX 77034",NaN,http://jobview.monster.com/Project-Manager-192...,NaN,Business/Strategic Management,ac2da666b68583ed3c3cc4eaee869eb1
6057,United States of America,US,NaT,No,jobs.monster.com,Associate is responsible for delivering materi...,Pick-Up Truck Driver ( D ),Full Time Employee,"Rochester, NY 14623",NaN,http://jobview.monster.com/pick-up-truck-drive...,NaN,Installation/Maintenance/Repair,2afee5e7f01a4a0d3723fb9f543f439e


In [202]:
df[df['job_title'].str.contains('\(....\)')].head(2)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
97,United States of America,US,NaT,No,jobs.monster.com,Job Title: Personal Banker (SAFE) 1 - Bear Val...,Personal Banker (SAFE) Bear Valley,Full Time Employee,"Denver, CO",Banking,http://jobview.monster.com/Personal-Banker-SAF...,NaN,Experienced (Non-Manager),c639b20ed01b45b3f3aff1ef58a5d9ce
198,United States of America,US,NaT,No,jobs.monster.com,Job Title: Personal Banker (SAFE) 1 - Cypress ...,Personal Banker (SAFE) Cypress Area Overhire,Full Time Employee,"Houston, TX",Banking,http://jobview.monster.com/Personal-Banker-SAF...,NaN,Experienced (Non-Manager),ce07a03b6d309d11388fd87c20b91e2e


In [203]:
df['job_title'] = df["job_title"].str.replace(r'\(....\)|\( . \)|\(\)',"",regex=True)

In [204]:
df.job_title

0                                IT Support Technician 
1                             Business Reporter/Editor 
2                               Senior Training Leader 
3                                   Engineer - Quality 
4                                  Shift Supervisor -  
                              ...                      
21995            Assistant Vice President - Controller 
21996                                       Accountant 
21997                                 AEM/CQ developer 
21998    Electrician - Experienced Forging Electrician 
21999                           Contract Administrator 
Name: job_title, Length: 22000, dtype: object

## [5] Changing Improper Uppercase Format To Consistent Format
So, giving proper format to job_description and job_title

In [205]:
df[df['job_title'].str.isupper()].head(3)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
31,United States of America,US,NaT,No,jobs.monster.com,RECREATION THERAPY ASSISTANT The successful a...,RECREATION THERAPY ASSISTANT,Full Time,"Knoxville, TN",NaN,http://jobview.monster.com/RECREATION-THERAPY-...,NaN,NaN,ec83b57fe35bc66e0a1a1cf4e65b3030
56,United States of America,US,NaT,No,jobs.monster.com,DescriptionIf your goal is to begin a fantasti...,CASHIER,Full Time,"Houston, TX",Retail,http://jobview.monster.com/CASHIER-Job-Houston...,NaN,Experienced (Non-Manager),bcf70c41acf97ac8024c0b38c6fcaf87
113,United States of America,US,NaT,No,jobs.monster.com,TEMPORARY STUDENT PARAPROFESSIONAL Paid Intern...,TEMPORARY STUDENT PARAPROFESSIONAL,Full Time,"West Palm Beach, FL 33431",All,http://jobview.local-jobs.monster.com/TEMPORAR...,NaN,NaN,841a58c886ae3910f95f61049ecba82f


In [206]:
df['job_title'] = df.job_title.str.title() # Converting to Title Format 
df['job_description'] = df.job_description.str.title()

In [207]:
df[df['job_title'].str.isupper()].head(3) #no rows of job_title with uppercase

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id


## No Cleaning for Country Column

In [208]:
df.country.isnull().sum() # Has no null values 

0

In [209]:
df.country.unique() # Only value is 'United States of America' 

array(['United States of America'], dtype=object)

## No Cleaning for Country_code Column

In [210]:
df.country_code.isnull().sum() # Has no null values 

0

In [211]:
df.country_code.unique() # Only value is 'US'

array(['US'], dtype=object)

## Cleaning for date_added Column

In [212]:
df.shape

(22000, 14)

In [213]:
df.date_added.notnull().sum() # out of 22000 rows only 122 rows are not null

122

In [214]:
df.date_added.unique()  

array([                          'NaT', '2016-05-10T00:00:00.000000000',
       '2016-05-13T00:00:00.000000000', '2016-05-09T00:00:00.000000000',
       '2016-06-10T00:00:00.000000000', '2017-01-02T00:00:00.000000000',
       '2016-06-17T00:00:00.000000000', '2016-06-24T00:00:00.000000000',
       '2016-06-23T00:00:00.000000000', '2016-12-20T00:00:00.000000000',
       '2016-12-12T00:00:00.000000000', '2016-07-15T00:00:00.000000000',
       '2016-07-08T00:00:00.000000000', '2016-10-31T00:00:00.000000000',
       '2016-10-28T00:00:00.000000000', '2016-04-28T00:00:00.000000000',
       '2016-04-27T00:00:00.000000000', '2016-04-29T00:00:00.000000000',
       '2016-09-12T00:00:00.000000000', '2017-01-16T00:00:00.000000000',
       '2016-09-22T00:00:00.000000000', '2016-03-25T00:00:00.000000000',
       '2016-06-20T00:00:00.000000000', '2016-11-17T00:00:00.000000000',
       '2016-07-11T00:00:00.000000000', '2016-12-07T00:00:00.000000000',
       '2016-12-14T00:00:00.000000000', '2016-12-29

#### [1]  Providing Consistent Format To Dates
Some dates are in mm/dd/yyyy format others are in m/d/yyyy format  ( '05/09/2016'  '1/2/2017')

In [215]:
df['date_added'] = df['date_added'].dt.strftime('%m-%d-%Y')

In [216]:
df.date_added.unique() 

array([nan, '05-10-2016', '05-13-2016', '05-09-2016', '06-10-2016',
       '01-02-2017', '06-17-2016', '06-24-2016', '06-23-2016',
       '12-20-2016', '12-12-2016', '07-15-2016', '07-08-2016',
       '10-31-2016', '10-28-2016', '04-28-2016', '04-27-2016',
       '04-29-2016', '09-12-2016', '01-16-2017', '09-22-2016',
       '03-25-2016', '06-20-2016', '11-17-2016', '07-11-2016',
       '12-07-2016', '12-14-2016', '12-29-2016', '05-05-2016',
       '05-06-2016', '08-11-2016', '03-17-2016', '04-04-2016',
       '04-07-2016', '04-06-2016', '09-21-2016', '05-18-2016',
       '03-15-2016', '05-19-2016', '06-08-2016', '06-07-2016',
       '10-18-2016', '12-02-2016', '11-07-2016', '03-29-2016',
       '04-08-2016', '05-26-2016', '09-15-2016', '08-17-2016',
       '11-21-2016', '12-15-2016', '10-27-2016', '10-25-2016',
       '06-27-2016', '07-05-2016', '07-04-2016', '07-07-2016',
       '08-25-2016', '09-05-2016', '08-23-2016', '09-02-2016',
       '09-01-2016', '08-31-2016', '09-26-2016', '

## No Cleaning for has_expired Column

In [217]:
df.has_expired.isnull().sum() 

0

In [218]:
df.has_expired.unique()  # No job posting add is expired.

array(['No'], dtype=object)

## No Cleaning for job_board Column

In [219]:
df.job_board.isnull().sum()

0

In [220]:
df.job_board.unique()

array(['jobs.monster.com'], dtype=object)

## Cleaning Job_Type Column

In [221]:
df.job_type.isnull().sum()

1628

In [222]:
df.job_type.unique() 
# It will have Job_type column storing (part time, full time), IsEmployee(if hiring is for employee or not), 
# IsIntern(if hiring is for intern or not), Job_Term
# Full Time for values
    # Full Time
    # Full Time, Employee
    # Full Time / Employee,etc
    
# Part Time for values
    # Part Time, Employee,etc
    
# IsEmployee = True for values : 
    # Full Time, Employee
    # Full Time Employee
    # Part Time Employee
    # Part Time, Employee,etc
    
# IsIntern = True for value 
    # Full Time Intern,etc
    
# Job_Term = Contract_based for values :
    # Temporary/Contract/Project
    # exempt
    # Seasonal
# nan 

array(['Full Time Employee', 'Full Time', 'Full Time, Employee',
       'Part Time Employee', nan, 'Full Time Temporary/Contract/Project',
       'Full Time , Employee', 'Full Time, Temporary/Contract/Project',
       'Employee', 'Part Time', 'Part Time, Employee', 'Full Time Intern',
       'Temporary/Contract/Project', 'Full Time / Employee',
       'Full Time , Temporary/Contract/Project',
       'Part Time, Temporary/Contract/Project', 'Full Time/ Employee',
       'Per Diem, Employee', 'Job Type Full Time Employee', 'Per Diem',
       'Full Time\xa0', 'Part Time Intern', 'Per Diem Employee',
       'Part Time/ Temporary/Contract/Project',
       'Part Time Temporary/Contract/Project', 'Exempt',
       'Part Time , Temporary/Contract/Project', 'Full Time\xa0 Employee',
       'Part Time Seasonal', 'Part Time , Employee', 'Job Type Employee',
       'Job Type Full Time Temporary/Contract/Project',
       'Full Time / > Employee', 'Part Time\xa0',
       'Per Diem, Temporary/Contract

#### [1] Create a new column IsEmployee and wherever Job_type contains "employee"  store 'True' in  IsEmployee else store 'False'

In [223]:
df['IsEmployee'] = df['job_type'].str.contains('Employee',na=False)

In [224]:
df[['job_type','IsEmployee']][df['IsEmployee'] == True].head(2)

,job_type,IsEmployee
0,Full Time Employee,True
2,"Full Time, Employee",True


#### [2] Create a new column IsIntern and wherever value contains "intern" in Job_type column store 'True' in IsIntern else store 'False'

In [225]:
df[df['job_type'].str.contains('Intern',na=False)].head(2)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id,IsEmployee
178,United States of America,US,NaN,No,jobs.monster.com,Springer Nature Is A Major New Force In Sc...,Editorial Internship,Full Time Intern,"New York, NY 10013",Printing and Publishing,http://jobview.monster.com/Editorial-Internshi...,NaN,Editorial/Writing,0405aaa87456bb11407f780bd5688132,False
2005,United States of America,US,NaN,No,jobs.monster.com,Summer Intern Ramjob Function: Salesprimary Lo...,Summer Marketing Internship,Part Time Intern,"Columbus, OH",NaN,http://jobview.monster.com/summer-marketing-in...,0.00 - 15.00 $ /hour,Marketing/Product,c83fe0d1f4cf4fa947910e7bd3de3563,False


In [226]:
df['IsIntern'] = df['job_type'].str.contains('Intern',na=False)

In [227]:
df[['job_type','IsIntern']][df['IsIntern'] == True].head(2)

,job_type,IsIntern
178,Full Time Intern,True
2005,Part Time Intern,True


#### [3] Create a new column JobTerm and wherever value contains "Temporary/Contract/Project", 'Seasonal'  in Job_type column store 'Contract_based'  else store 'Unknown'

In [228]:
df['JobTerm'] =np.where(df['job_type'].str.contains(r'Temporary/Contract/Project|Seasonal',na=False,regex=True),'Contract_based','Unknown')

In [229]:
df[['job_type','JobTerm']][df['job_type'].str.contains('Seasonal',na=False)].head(2)

,job_type,JobTerm
7833,Part Time Seasonal,Contract_based
15923,Part Time Seasonal,Contract_based


In [230]:
df[['job_type','JobTerm']][df['job_type'].str.contains('Temporary/Contract/Project',na=False)].head(2)

,job_type,JobTerm
21,Full Time Temporary/Contract/Project,Contract_based
30,Full Time Temporary/Contract/Project,Contract_based


#### [4] Removing Info like "Employee",'Intern','seasonal',etc from job_type

In [231]:
df[df['job_type'].str.contains("Employee",na=False)].head(2)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id,IsEmployee,IsIntern,JobTerm
0,United States of America,US,NaN,No,jobs.monster.com,Teamsoft Is Seeing An It Support Specialist To...,It Support Technician,Full Time Employee,"Madison, WI 53702",NaN,http://jobview.monster.com/it-support-technici...,NaN,IT/Software Development,11d599f229a80023d2f40e7c52cd941e,True,False,Unknown
2,United States of America,US,NaN,No,jobs.monster.com,Report This Job About The Job Depuy Synthes Co...,Senior Training Leader,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,http://jobview.monster.com/senior-training-lea...,NaN,NaN,839106b353877fa3d896ffb9c1fe01c0,True,False,Unknown


In [232]:
df[df['job_type'].str.contains("Intern",na=False)]

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id,IsEmployee,IsIntern,JobTerm
178,United States of America,US,NaN,No,jobs.monster.com,Springer Nature Is A Major New Force In Sc...,Editorial Internship,Full Time Intern,"New York, NY 10013",Printing and Publishing,http://jobview.monster.com/Editorial-Internshi...,NaN,Editorial/Writing,0405aaa87456bb11407f780bd5688132,False,True,Unknown
2005,United States of America,US,NaN,No,jobs.monster.com,Summer Intern Ramjob Function: Salesprimary Lo...,Summer Marketing Internship,Part Time Intern,"Columbus, OH",NaN,http://jobview.monster.com/summer-marketing-in...,0.00 - 15.00 $ /hour,Marketing/Product,c83fe0d1f4cf4fa947910e7bd3de3563,False,True,Unknown
2422,United States of America,US,NaN,No,jobs.monster.com,Position Summary The Hr Co-Op Will Be A Part O...,Hr Internship/Co-Op,Full Time Intern,"Algona, WA 98001",Aerospace and Defense,http://jobview.monster.com/hr-internship-co-op...,NaN,Human Resources,d99d552308bec9a14c50e8b1e9777941,False,True,Unknown
11347,United States of America,US,NaN,No,jobs.monster.com,Essential Functions: Monitors The It Suppor...,Information Systems Intern,"Part Time, Intern","Columbus, OH",Energy and Utilities,http://jobview.monster.com/information-systems...,NaN,IT/Software Development,ad9799eee47b0d7a95d806216860c8a5,False,True,Unknown
17079,United States of America,US,NaN,No,jobs.monster.com,"Our Mission: “At Bbrg, We Strive To Be The Be...",Digital Marketing Intern,Part Time Intern,"Columbus, OH 43085",Restaurant/Food Services,http://jobview.monster.com/Digital-Marketing-I...,NaN,Entry Level,25eb1c21490394f643a041500ddbd754,False,True,Unknown
17959,United States of America,US,02-29-2016,No,jobs.monster.com,Lehigh Hanson Is Seeking An Intern – Informati...,Front Office It Intern,Full Time Intern,"Irving, TX 75062",Metals and MineralsConstruction - Industrial F...,http://jobview.monster.com/Front-Office-IT-Int...,NaN,IT/Software Development,de1ab6c78ef1b828239b79200d308801,False,True,Unknown
19916,United States of America,US,NaN,No,jobs.monster.com,"Overview: Aesculap, Inc., A B. Braun Company, ...",Marketing Internship,Full Time Intern,"Center Valley, PA",Medical Devices and Supplies,http://jobview.monster.com/Marketing-Internshi...,NaN,Medical/Health,8418b7d1135191bc41bd50db104f4697,False,True,Unknown
20137,United States of America,US,NaN,No,jobs.monster.com,Brand Ambassador Internship As A Brand Ambassa...,Brand Ambassador Internship,Full Time Intern,"Wauwatosa, WI 53222",Advertising and PR Services,http://jobview.monster.com/brand-ambassador-in...,NaN,Marketing/Product,889d830fcba9c1392090c2cf79b821cd,False,True,Unknown


In [233]:
df['job_type'] = df["job_type"].str.replace(r'((?i)Employee|(?i)Intern|(?i)Temporary/Contract/Project|(?i)Seasonal)','',regex=True)

#### [5] Changing Value called 'Per Diem'(Latin Word) In job_type column to 'As Required' (English Transaltion)
'Per Diem' Is A Latin Word Which Means “as needed” and does not guarantee hours, usually working less than full time but not necesarry to be part time

In [234]:
df[df['job_type'].str.contains("Per Diem",na=False)].head(2)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id,IsEmployee,IsIntern,JobTerm
364,United States of America,US,NaN,No,jobs.monster.com,Provides Respiratory Therapy Modalities Per Pr...,P/D Advanced Respiratory Care Practitioner,"Per Diem,","Las Vegas, NV",Healthcare Services,http://jobview.monster.com/p-d-advanced-respir...,NaN,Medical/Health,2e39a794c0a3ac1f15dc7072d3cb5521,True,False,Unknown
610,United States of America,US,NaN,No,jobs.monster.com,Management Reserves The Right To Change The Es...,Security Officer (Oc),Per Diem,"Rochester, WA 98579",Security and Surveillance,http://jobview.monster.com/security-officer-oc...,NaN,Security/Protective Services,eec383a41686cdc5dddb60a598065231,False,False,Unknown


In [235]:
df['job_type'] = df["job_type"].str.replace("Per Diem","As Required") 

In [236]:
df[df['job_type'].str.contains("As Required",na=False)].head(2)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id,IsEmployee,IsIntern,JobTerm
364,United States of America,US,NaN,No,jobs.monster.com,Provides Respiratory Therapy Modalities Per Pr...,P/D Advanced Respiratory Care Practitioner,"As Required,","Las Vegas, NV",Healthcare Services,http://jobview.monster.com/p-d-advanced-respir...,NaN,Medical/Health,2e39a794c0a3ac1f15dc7072d3cb5521,True,False,Unknown
610,United States of America,US,NaN,No,jobs.monster.com,Management Reserves The Right To Change The Es...,Security Officer (Oc),As Required,"Rochester, WA 98579",Security and Surveillance,http://jobview.monster.com/security-officer-oc...,NaN,Security/Protective Services,eec383a41686cdc5dddb60a598065231,False,False,Unknown


#### [6] Some Miscellaneous Changes In job_type

In [237]:
df.job_type.unique()

array(['Full Time ', 'Full Time', 'Full Time, ', 'Part Time ', nan,
       'Full Time , ', '', 'Part Time', 'Part Time, ', 'Full Time / ',
       'Full Time/ ', 'As Required, ', 'Job Type Full Time ',
       'As Required', 'Full Time\xa0', 'As Required ', 'Part Time/ ',
       'Exempt', 'Part Time , ', 'Full Time\xa0 ', 'Job Type ',
       'Full Time / > ', 'Part Time\xa0', 'Job Type Full Time',
       'Part Time / ', 'Job Type Part Time '], dtype=object)

In [238]:
df['job_title'] = df["job_title"].str.replace(r'Part Time , |Part Time \/ |Part Time\\xa0|Part Time\/ |Part Time, |Job Type Part Time |Part Time ',"Part Time",regex=True)

In [239]:
df['job_title'] = df["job_title"].str.replace(r'Full Time,|Full Time , |Full Time, |Full Time \/ |Full Time\/ |Job Type Full Time |Full Time\\xa0|Job Type Full Time|Full Time ',"Full Time",regex=True)

In [240]:
df.IsIntern.unique()

array([False,  True])

In [241]:
df.IsEmployee.unique()

array([ True, False])

In [242]:
df.JobTerm.unique()

array(['Unknown', 'Contract_based'], dtype=object)

## Cleaning Location Column

In [243]:
df.location

0                                        Madison, WI 53702
1                                        Madison, WI 53708
2        DePuy Synthes Companies is a member of Johnson...
3                                                Dixon, CA
4                                             Camphill, PA
                               ...                        
21995                                       Cincinnati, OH
21996                                 Cincinnati, OH 45236
21997                                    Chicago, IL 60602
21998                                    Chicago, IL 60609
21999                                       Cincinnati, OH
Name: location, Length: 22000, dtype: object

In [244]:
# city, abbrevated state name , Pincode
# few location should be swaped from organization

In [245]:
df['location'].str.split("Location",n=1,expand = True)

,0,1
0,"Madison, WI 53702",None
1,"Madison, WI 53708",None
2,DePuy Synthes Companies is a member of Johnson...,United States-Massachusetts-RaynhamOrganizatio...
3,"Dixon, CA",None
4,"Camphill, PA",None
...,...,...
21995,"Cincinnati, OH",None
21996,"Cincinnati, OH 45236",None
21997,"Chicago, IL 60602",None
21998,"Chicago, IL 60609",None


In [246]:
df[['location','Location_dummy']] = df['location'].str.split("Location",n=1,expand = True)

In [247]:
df['location'] = df['location'].mask(df['Location_dummy'].notnull(),df['Location_dummy'],axis = 0)

In [248]:
df[['location','Location_dummy']].head(10)

,location,Location_dummy
0,"Madison, WI 53702",None
1,"Madison, WI 53708",None
2,United States-Massachusetts-RaynhamOrganizatio...,United States-Massachusetts-RaynhamOrganizatio...
3,"Dixon, CA",None
4,"Camphill, PA",None
5,"Charlottesville, VA",None
6,Contact name Tony Zerio,None
7,"Austin, TX 73301",None
8,"Austin, TX 78746",None
9,"Chesterfield, MO",None


In [249]:
df.drop("Location_dummy",axis=1,inplace = True)

In [250]:
df.location

0                                        Madison, WI 53702
1                                        Madison, WI 53708
2        United States-Massachusetts-RaynhamOrganizatio...
3                                                Dixon, CA
4                                             Camphill, PA
                               ...                        
21995                                       Cincinnati, OH
21996                                 Cincinnati, OH 45236
21997                                    Chicago, IL 60602
21998                                    Chicago, IL 60609
21999                                       Cincinnati, OH
Name: location, Length: 22000, dtype: object

In [251]:
# When organization value is location store index value of it
idx = df.organization.isin(list(df.location))

In [252]:
# swap values between organization and location for index vales stored in idx
df.loc[idx,['location','organization']] = df.loc[idx,['organization','location']].values
df.loc[idx,['location','organization']]

,location,organization
29,"Chicago, IL","Sr. Process Engineer, Manufacturing"
36,"Oklahoma City, OK","RF System Technician, Field Service"
38,"San Francisco, CA",Bi-Lingual Editorial Strategist
73,"Durham, NC",Quality Engineer
152,"Houston, TX",Business Analyst
...,...,...
21401,"Cincinnati, OH",Performance Engineer
21456,"Cincinnati, OH",Supply Chain Business Analyst
21507,"Cincinnati, OH",Project Coordinator
21629,"Cincinnati, OH",Help Desk / Desktop Support Technician


In [253]:
df['location'].str.split(",",n=1,expand = True)

,0,1
0,Madison,WI 53702
1,Madison,WI 53708
2,United States-Massachusetts-RaynhamOrganizatio...,None
3,Dixon,CA
4,Camphill,PA
...,...,...
21995,Cincinnati,OH
21996,Cincinnati,OH 45236
21997,Chicago,IL 60602
21998,Chicago,IL 60609


In [254]:
df[['city','location']] = df['location'].str.split(",",n=1,expand = True)

In [255]:
df[['city','location']]

,city,location
0,Madison,WI 53702
1,Madison,WI 53708
2,United States-Massachusetts-RaynhamOrganizatio...,None
3,Dixon,CA
4,Camphill,PA
...,...,...
21995,Cincinnati,OH
21996,Cincinnati,OH 45236
21997,Chicago,IL 60602
21998,Chicago,IL 60609


In [256]:
df[['city','drop']] = df['city'].str.split("Organization",n=1,expand = True)

In [257]:
df['city']

0                                    Madison
1                                    Madison
2        United States-Massachusetts-Raynham
3                                      Dixon
4                                   Camphill
                        ...                 
21995                             Cincinnati
21996                             Cincinnati
21997                                Chicago
21998                                Chicago
21999                             Cincinnati
Name: city, Length: 22000, dtype: object

In [258]:
df['city'] = df['city'].replace("United States-Massachusetts-Raynham",'Raynham')

In [259]:
df['city']

0           Madison
1           Madison
2           Raynham
3             Dixon
4          Camphill
            ...    
21995    Cincinnati
21996    Cincinnati
21997       Chicago
21998       Chicago
21999    Cincinnati
Name: city, Length: 22000, dtype: object

In [260]:
df.location

0         WI 53702
1         WI 53708
2             None
3               CA
4               PA
           ...    
21995           OH
21996     OH 45236
21997     IL 60602
21998     IL 60609
21999           OH
Name: location, Length: 22000, dtype: object

In [261]:
df['location'].str[:3]

0          WI
1          WI
2        None
3          CA
4          PA
         ... 
21995      OH
21996      OH
21997      IL
21998      IL
21999      OH
Name: location, Length: 22000, dtype: object

In [262]:
df['state_code'] = df['location'].str[:3]

In [263]:
df.state_code

0          WI
1          WI
2        None
3          CA
4          PA
         ... 
21995      OH
21996      OH
21997      IL
21998      IL
21999      OH
Name: state_code, Length: 22000, dtype: object

In [264]:
df['pincode'] = df['location'].str[3:]

In [265]:
df.pincode

0         53702
1         53708
2          None
3              
4              
          ...  
21995          
21996     45236
21997     60602
21998     60609
21999          
Name: pincode, Length: 22000, dtype: object

## Cleaning Organization Column

In [266]:
# Organization_name  
# some has location values
# other/not classified/All  -> Unknowm

In [267]:
df[df['organization'].str.contains(r'All\b',na=False)].head(2)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,...,salary,sector,uniq_id,IsEmployee,IsIntern,JobTerm,city,drop,state_code,pincode
57,United States of America,US,NaN,No,jobs.monster.com,Enterprise Products Partners L.P. Is One Of ...,Specialist,Full Time,TX 77001,All,...,NaN,Experienced (Non-Manager),54b3bb1da783d46a5b4db82995ff357f,False,False,Unknown,Houston,None,TX,77001
113,United States of America,US,NaN,No,jobs.monster.com,Temporary Student Paraprofessional Paid Intern...,Temporary Student Paraprofessional,Full Time,FL 33431,All,...,NaN,NaN,841a58c886ae3910f95f61049ecba82f,False,False,Unknown,West Palm Beach,None,FL,33431


In [268]:
df[df['organization'].str.contains(r'^Other/Not Classified',na=False)].head(2)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,...,salary,sector,uniq_id,IsEmployee,IsIntern,JobTerm,city,drop,state_code,pincode
42,United States of America,US,NaN,No,jobs.monster.com,Job Description:Job Summarythe Office Administ...,Office Administrator -,Full Time,CO 80237,Other/Not Classified,...,Up to $32000.00,Experienced (Non-Manager),1f2da47e60173c6667395f081c048713,False,False,Unknown,Denver,None,CO,80237
69,United States of America,US,NaN,No,jobs.monster.com,Do You Have A Passion For The Beauty Industry?...,Salon Coordinator,Part Time,ND,Other/Not Classified,...,NaN,Customer Support/Client Care,2f9e2894fbff2bddd6864444b9a81e75,True,False,Unknown,Fargo,None,ND,


In [269]:
df['organization'] = df["organization"].str.replace(r'^Other/Not Classified|All\b',"Unknown",regex=True)

#### Some Values Are Of Location So, Swapping Them

In [270]:
df.loc[29,'organization'] 

'Sr. Process Engineer, Manufacturing'

In [271]:
idx = df.organization.isin(list(df.location)) 
df.loc[idx,['location','organization']] = df.loc[idx,['organization','location']].values

In [272]:
df['organization']

0                                                   NaN
1                               Printing and Publishing
2                       Personal and Household Services
3                                      Altec Industries
4                                                Retail
                              ...                      
21995                                               NaN
21996    Construction - Residential & Commercial/Office
21997                                               NaN
21998                         Jernberg Industries, Inc.
21999                                               NaN
Name: organization, Length: 22000, dtype: object

## Cleaning page_url Column

In [273]:
df.page_url.notnull().sum() # no null

22000

In [274]:
df.page_url.unique()

array(['http://jobview.monster.com/it-support-technician-job-madison-wi-us-167855963.aspx?mescoid=1500134001001&jobPosition=20',
       'http://jobview.monster.com/business-reporter-editor-job-madison-wi-us-167830105.aspx?mescoid=2700437001001&jobPosition=7',
       'http://jobview.monster.com/senior-training-leader-job-raynham-ma-us-177958678.aspx?mescoid=1100055001001&jobPosition=4',
       ...,
       'http://jobview.monster.com/AEM-CQ5-developer-Job-chicago-IL-US-165384644.aspx?mescoid=1500127001001&jobPosition=1',
       'http://jobview.monster.com/Electrician-Experienced-Forging-Electrician-Job-Chicago-IL-US-160653991.aspx?mescoid=4700805001001&jobPosition=16',
       'http://jobview.monster.com/Contract-Administrator-Job-Cincinnati-OH-US-164541784.aspx?mescoid=1300093001001&jobPosition=5'],
      dtype=object)

## Cleaning Sector Column

In [288]:
df.sector

0                      IT/Software Development
1                                          NaN
2                                          NaN
3                    Experienced (Non-Manager)
4                   Project/Program Management
                         ...                  
21995                                      NaN
21996    Manager (Manager/Supervisor of Staff)
21997                                      NaN
21998          Installation/Maintenance/Repair
21999                Experienced (Non-Manager)
Name: sector, Length: 22000, dtype: object

In [276]:
#has sector name
#take some values from organization name
#has some values of experience or requirements or prerquisites values

In [277]:
df[df['sector'].str.contains('Experienced',na=False)].head(2)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,...,salary,sector,uniq_id,IsEmployee,IsIntern,JobTerm,city,drop,state_code,pincode
3,United States of America,US,NaN,No,jobs.monster.com,Why Join Altec? If You’Re Considering A Career...,Engineer - Quality,Full Time,CA,Altec Industries,...,NaN,Experienced (Non-Manager),58435fcab804439efdcaa7ecca0fd783,False,False,Unknown,Dixon,None,CA,
5,United States of America,US,NaN,No,jobs.monster.com,Job Description Job #: 720298Apex Systems Has...,Construction Pm - Charlottesville,Full Time,VA,Computer/IT Services,...,NaN,Experienced (Non-Manager),1e2637cb5f7a2c4615a99a26c0566c66,True,False,Unknown,Charlottesville,None,VA,


In [278]:
df[df['sector'].str.contains('Entry Level',na=False)].head(2)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,...,salary,sector,uniq_id,IsEmployee,IsIntern,JobTerm,city,drop,state_code,pincode
13,United States of America,US,NaN,No,jobs.monster.com,Launch Your Teaching Career With The Leader In...,Primrose Private Preschool Teacher,Full Time,TX 77098,Education,...,9.00 - 13.00 $ /hour,Entry Level,b43c077756d5a326c4854e1399fd2464,False,False,Unknown,Houston,None,TX,77098
46,United States of America,US,NaN,No,jobs.monster.com,"Adolfson & Peterson (Ap), One Of The Nation’S ...",Project Engineer- Co,Full Time,CO 80011,Construction - Residential & Commercial/Office,...,NaN,Entry Level,218b1fcde8264f6aa454aa823aa13780,False,False,Unknown,Aurora,None,CO,80011


In [279]:
df['Experienced_level'] = df['sector'].mask(df['sector'].str.contains('Experienced').notnull(),'True',axis=0)

In [280]:
df['Beginner_level'] = df['sector'].mask(df['sector'].str.contains('Entry Level').notnull(),'True',axis=0)

## No Cleaning For uniq_id 

In [281]:
df.uniq_id.notnull().sum() #no null values

22000

In [282]:
df.uniq_id.duplicated().sum() # all unique values

0

## Final Cleaning

In [283]:
df.head(2)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,...,uniq_id,IsEmployee,IsIntern,JobTerm,city,drop,state_code,pincode,Experienced_level,Beginner_level
0,United States of America,US,NaN,No,jobs.monster.com,Teamsoft Is Seeing An It Support Specialist To...,It Support Technician,Full Time,WI 53702,NaN,...,11d599f229a80023d2f40e7c52cd941e,True,False,Unknown,Madison,None,WI,53702,True,True
1,United States of America,US,NaN,No,jobs.monster.com,The Wisconsin State Journal Is Seeking A Flexi...,Business Reporter/Editor,Full Time,WI 53708,Printing and Publishing,...,e4cbb126dabf22159aff90223243ff2a,False,False,Unknown,Madison,None,WI,53708,NaN,NaN


### Lets Drop Unnecessary Columns

#### Country, Country code, has_expiired,job_board : (These Can be dropped if needed as they contain only one unique value each column, but as it depends on requirements i am leaving them as it is and dropping only those columns which have repeated information)

#### Dropping columns : location( as i have divided location into city state and pincode), drop(this was a dummy column i have created)

In [284]:
df.drop("location",axis=1,inplace = True)
df.drop("drop",axis=1,inplace = True)

In [285]:
df.head(2)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,organization,page_url,...,sector,uniq_id,IsEmployee,IsIntern,JobTerm,city,state_code,pincode,Experienced_level,Beginner_level
0,United States of America,US,NaN,No,jobs.monster.com,Teamsoft Is Seeing An It Support Specialist To...,It Support Technician,Full Time,NaN,http://jobview.monster.com/it-support-technici...,...,IT/Software Development,11d599f229a80023d2f40e7c52cd941e,True,False,Unknown,Madison,WI,53702,True,True
1,United States of America,US,NaN,No,jobs.monster.com,The Wisconsin State Journal Is Seeking A Flexi...,Business Reporter/Editor,Full Time,Printing and Publishing,http://jobview.monster.com/business-reporter-e...,...,NaN,e4cbb126dabf22159aff90223243ff2a,False,False,Unknown,Madison,WI,53708,NaN,NaN


### Rearranging Columns In Better Sequence

In [286]:
df = df.reindex(columns= ['job_title' ,'organization','sector','job_type','IsEmployee','IsIntern','Experienced_level','Beginner_level','job_description','country','country_code','state_code','city','pincode','date_added','has_expired','job_board','page_url','uniq_id' ])

In [287]:
df.head(2)

,job_title,organization,sector,job_type,IsEmployee,IsIntern,Experienced_level,Beginner_level,job_description,country,country_code,state_code,city,pincode,date_added,has_expired,job_board,page_url,uniq_id
0,It Support Technician,NaN,IT/Software Development,Full Time,True,False,True,True,Teamsoft Is Seeing An It Support Specialist To...,United States of America,US,WI,Madison,53702,NaN,No,jobs.monster.com,http://jobview.monster.com/it-support-technici...,11d599f229a80023d2f40e7c52cd941e
1,Business Reporter/Editor,Printing and Publishing,NaN,Full Time,False,False,NaN,NaN,The Wisconsin State Journal Is Seeking A Flexi...,United States of America,US,WI,Madison,53708,NaN,No,jobs.monster.com,http://jobview.monster.com/business-reporter-e...,e4cbb126dabf22159aff90223243ff2a
